## 1. Importing Libraries and Setup Google Drive

In [ ]:
!pip install bitsandbytes
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
def get_prompt(prompt_name:str) -> str:
    """
    Set the prompt the model we'll see as input.
    We defined three types of prompts

    Arguments:

    - prompt_name: represents the prompt of a page, it must be a string


    """
    prompt = ""
    match prompt_name:
        case "in_context":
            prompt = """Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo. Di seguito alcuni esempi:

            Testo antico: quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
            Traduzione moderna: Quella guerra fu ben condotta per raggiungere il suo scopo. Dall'altra parte, Aiace era un cavaliere leale e valoroso nelle armi, di grande statura, ma non molto saggio.

            Testo antico: crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
            Traduzione moderna: È crudele e si vendica di ogni colpa, come stabilisce la legge, e non perdona alcun cavaliere che commetta un errore.

            Testo antico: Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
            Traduzione moderna: Ponzio Aufidiano, cavaliere romano, non era dotato di un coraggio superiore.

            Testo antico: Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
            Traduzione moderna: Se questo è il volere di tutti, e se i tempi richiedono Pompeo come guida e non come compagno, allora non tratterrò oltre il destino.

            Testo antico: Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire.
            Traduzione moderna: Il compito di quest’arte sembra essere quello di parlare in modo studiato per convincere; il fine è dunque persuadere attraverso le parole.

            Perfavore non includere nella risposta il testo precedente, ma soltanto la traduzione.
            Traduci la frase da italiano antico a quello moderno, scrivendo solo ed ESCLUSIVAMENTE la risposta.
            Testo antico: {sentence}
            Traduzione moderna:
            """
        case "language_expert":
            prompt = """Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo.
            Perfavore non includere nella risposta il testo precedente, ma soltanto la traduzione.
            Traduci la seguente frase da italiano antico a quello moderno, scrivendo solo ed ESCLUSIVAMENTE la risposta.
            Testo antico: {sentence}
            Traduzione moderna:
            """
        case "with_rules":
            prompt = """Traduci e modernizza una frase in italiano contemporaneo e comprensibile, data una frase in italiano Antico.
            Fai affidamento in TUTTA la CONOSCENZA POSSEDUTA nel tradurre e modernizzare frasi da Italiano Antico a Italiano Moderno.

            Ecco un esempio di traduzione:
              Testo antico: Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
              Traduzione moderna: Ponzio Aufidiano, cavaliere romano, non era dotato di un coraggio superiore.

            Devi seguire le seguenti regole:
                1. La frase prodotta deve essere comprensibile
                2. La frase prodotta deve avere un senso
                3. Ristruttura la frase, in modo che sia più contemporanea possibile
                4. Tutte le parole devono essere in italiano contemporaneo, escludendo nomi o luoghi

            Perfavore restuituisci SOLO ED ESCLUSIVAMENTE la la frase tradotta e modernizzata.
            Testo antico: {sentence}
            Traduzione moderna:
            """
        case _:
            raise Exception("Unexpected name")
            exit(1)

    return prompt


## 2. Option for the Quantized version of the model

In [ ]:
quantize_model = False

"""
You can choose from:
    1. "in_context"
    2. "language_expert"
    3. "with_rules"


"""
prompt = get_prompt("in_context")
prompt

"Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo. Di seguito alcuni esempi:\n\n            Testo antico: quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno\n            Traduzione moderna: Quella guerra fu ben condotta per raggiungere il suo scopo. Dall'altra parte, Aiace era un cavaliere leale e valoroso nelle armi, di grande statura, ma non molto saggio.\n\n            Testo antico: crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.\n            Traduzione moderna: È crudele e si vendica di ogni colpa, come stabilisce la legge, e non perdona alcun cavaliere che commetta un errore.\n\n            Testo antico: Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.\n            Traduzione moderna: Ponzio Aufidiano, cavaliere romano, non era dot

In [ ]:
from transformers import BitsAndBytesConfig

quant_config = None
if quantize_model:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

## 3. Loading Test Set

In [6]:
df = pd.read_csv("test.csv", sep=";")
df = df.rename(columns={"Sentence": "source", "Traductions": "target"})

dataset = Dataset.from_pandas(df)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## 4. Define Quantization and Load Model

In [ ]:
model_name = "google/flan-t5-XL"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                              device_map="auto",
                                              quantization_config=quant_config)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

## 5. Prometheus Evaluator
Since the PROMETHEUS model used with VLLM has 7 billion parameters, we attempted to load it using
the Hugging Face Transformers library and then quantize it in order to reduce memory usage and improve inference efficiency.   
To address this:

 - We used the Hugging Face Transformers library to load the model, as it provides a standardized interface for accessing pretrained weights and integrating them into existing pipelines.

 - We then applied quantization, a common technique that reduces the numerical precision of the model weights , with the goal of:

        - Lowering memory consumption

        - Speeding up inference

        - Maintaining reasonable accuracy

In [ ]:
from prometheus import PrometheusEval_AtM

evaluator = PrometheusEval_AtM(quantized = True, device = device)

## 6. Translating phase

In [ ]:
import re
i = 1

source_sentences = []
predicted_sentences = []
gold_sentences = [] #annoted by hand
prometheus_score = []
df_col = ["source_sentences", "gold_sentences","predicted_sentences", "prometheus_score","GPT_score","user_score"]

print(len(dataset))
for sample in dataset:


    input_sentence = sample["source"]
    target_sentence = sample["target"]

    author = sample["Author"]
    date = sample["Date"]
    region = sample["Region"]

    user_prompt = prompt.format(sentence=input_sentence)

    translation =  pipe(user_prompt, max_new_tokens=512)[0]['generated_text']

    evaluation = evaluator.getEvaluation(input_sentence, translation, target_sentence)


    match_ = re.search(r'\[RESULT\]\s*(\d)', evaluation)
    if match_:
      result = int(match_.group(1))

    else:
      result = 0

    source_sentences.append(input_sentence)
    predicted_sentences.append(translation)
    gold_sentences.append(target_sentence)
    prometheus_score.append(result)



    print(f"Sentence {i}")
    print(f"\tItaliano Arcaico            -> {input_sentence}")
    print(f"\tItaliano moderno            -> {translation}")
    print(f"\tGOLD LABEL                  -> {target_sentence}")
    print(f"\tPROMETHEUS EVALUATION       -> {result}")
    print(f"-----------------------------------------")
    i+=1

z = [0 for _ in range(len(dataset))]
GPT_score,user_score = z,z
df = pd.DataFrame(list(zip(source_sentences, gold_sentences,predicted_sentences,prometheus_score,GPT_score,user_score)), columns=df_col)

df.to_csv("test_results_base.csv", sep=";")